In [200]:
import pandas as pd

from collections import OrderedDict
from dataclasses import dataclass, field

import altair as alt


In [201]:
df = pd.read_csv("./data/example_account_v2.csv")
df.head()

,tx_block_number,tx_index,tx_block_time,tx_hash,sender,receiver,symbol,contract_address,amount,project,token_bought_symbol,token_bought_amount,token_sold_symbol,token_sold_amount,tx_type,usd_price
0,11388748.0,NaN,2020-12-04 21:59:46.000 UTC,0x4599c62b2445000c777116e76c0ec2195cd3b0328d47...,0x87e6d8a8e4ea3478bb99a1a6ccfc4283074d948b,0xe47d5cc33517d5b8433ff249efe094b989989acc,ETH,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,586.850833
1,11388805.0,NaN,2020-12-04 22:14:12.000 UTC,0xd41d29f9b597cfeae2e23ca9429c273438341c18ac11...,0xe47d5cc33517d5b8433ff249efe094b989989acc,0x000000000000541e251335090ac5b47176af4f7e,ETH,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0.980000,NaN,NaN,NaN,NaN,NaN,NaN,578.751667
2,11390238.0,NaN,2020-12-05 03:37:14.000 UTC,0xbc8c6bfee9446c1df8d61acf4702cc24b5efaa71d71b...,0x000000000000541e251335090ac5b47176af4f7e,0xe47d5cc33517d5b8433ff249efe094b989989acc,ETH,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0.977286,NaN,NaN,NaN,NaN,NaN,NaN,576.750000
3,11400659.0,NaN,2020-12-06 18:02:59.000 UTC,0x35cbf08afdf02ffd33838636ca630ed3be3b2b5ae377...,0xe47d5cc33517d5b8433ff249efe094b989989acc,0xa36972e347e538e6c7afb9f44fb10dda7bba9ba2,ETH,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0.991995,NaN,NaN,NaN,NaN,NaN,NaN,595.811667
4,11779681.0,NaN,2021-02-02 23:10:53.000 UTC,0xccc80db83c50be641119c92bb40cabbf3b0ff88da245...,0xe5c405c5578d84c5231d3a9a29ef4374423fa0c2,0xe47d5cc33517d5b8433ff249efe094b989989acc,ETH,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0.323351,NaN,NaN,NaN,NaN,NaN,NaN,1522.431667


In [202]:
account = '0xe47d5cc33517d5b8433ff249efe094b989989acc'

@dataclass
class Token():
    '''
    Contains total information on token history
    '''
    
    token_symbol: str
    balance: OrderedDict = field(default_factory=OrderedDict) # Key is timestamp, value is balance
    debt: OrderedDict = field(default_factory=OrderedDict)
    tx_history: list = field(default_factory=list)

    def _get_previous_balance(self) -> float:
        '''Returns the previous balance'''
        
        if len(self.balance) == 0:
            balance = 0
        else:
            last_tx = next(reversed(OrderedDict(self.balance)))
            balance = self.balance[last_tx]
            
        return balance
    
    def _get_previous_debt(self) -> float:
        if len(self.debt) == 0:
            debt = 0
        else:
            last_tx = next(reversed(OrderedDict(self.debt)))
            debt = self.debt[last_tx]
        
        return debt
    
    def update_balance(self, timestamp, amount):
        '''
        
        '''
        self.balance[timestamp] = max(self._get_previous_balance() + amount, 0)
    
    def update_debt(self, timestamp, amount):
        self.debt[timestamp] = max(self._get_previous_debt() + amount, 0)
        

@dataclass
class Wallet():
    ''''
    Contains all the tokens
    '''
    
    tokens: dict = field(default_factory=dict)
    
    def init_token(self, token_symbol):
        if token_symbol not in self.tokens.keys():
            self._add_token(token_symbol)
    
    
    def update_token(self, token_symbol, timestamp, amount, tx_type, sender, receiver):
        
        self.init_token(token_symbol)
            
        token = self.get_token(token_symbol)

        
        if tx_type == 'borrow':
            token.update_debt(timestamp, amount)
        
        elif tx_type == 'repay':
            token.update_debt(timestamp, -amount)
    
        if sender == account:
            token.update_balance(timestamp, -amount)
            
        elif receiver == account:
            token.update_balance(timestamp, amount)

    def get_token(self, token_symbol):
        return self.tokens.get(token_symbol)
    
    def get_token_balance(self, token_symbol):
        
        return (pd.DataFrame(dict(self.tokens.get(token_symbol).balance).items(), columns=['timestamp', 'balance'])
                .assign(timestamp = lambda df_: pd.to_datetime(df_.timestamp)))
    
    def get_token_debt(self, token_symbol):
        
        return (pd.DataFrame(dict(self.tokens.get(token_symbol).debt).items(), columns=['timestamp', 'debt'])
                .assign(timestamp = lambda df_: pd.to_datetime(df_.timestamp)))
    
    def _add_token(self, token_symbol):
        
        self.tokens[token_symbol] = Token(token_symbol=token_symbol) 


In [203]:
df.loc[(df['receiver'] == '0xdcd33426ba191383f1c9b431a342498fdac73488') & (df.symbol == 'ETH'), 'tx_type'] = 'deposit'

In [217]:
wallet = Wallet()
swap_list = []
deposit_df = pd.DataFrame()

for index, row in df.drop_duplicates(subset=['tx_hash', 'symbol']).iterrows():
    
    wallet.init_token(row['symbol'])
    tx_hash = row['tx_hash']

    
    if tx_hash not in swap_list:
        swap_tx = check_swap(wallet, tx_hash)
        swap_list.append(swap_tx)
        
    wallet.update_token(row['symbol'], row['tx_block_time'], row['amount'], row['tx_type'], row['sender'], row['receiver'])
    
    if (row['tx_type'] == 'deposit') & (row['symbol'][0].lower() != 'a'):
        deposit_df = pd.concat([deposit_df, pd.DataFrame({'timestamp':row['tx_block_time'],
                                                          'symbol':row['symbol'], 
                                                          'deposit':row['amount'], 
                                                          'debt':wallet.get_token(row['symbol'])._get_previous_debt(),
                                                          'price':row['usd_price']}, index=[index])])
        

    
    

In [218]:
dfd = deposit_df.copy()
dfd['debt_deposit'] = dfd[['deposit', 'debt']].min(axis=1)
dfd['timestamp'] = pd.to_datetime(dfd.timestamp)
dfd['non_debt_deposit'] = dfd['deposit'] - dfd['debt_deposit']

dfd[['deposit', 'debt', 'debt_deposit', 'non_debt_deposit']] = dfd.groupby('symbol')[['deposit', 'debt','debt_deposit', 'non_debt_deposit']].cumsum()

In [219]:
coins = ['ETH', 'WBTC']
deposit_plot = dfd.query("symbol in @coins").melt(id_vars=['symbol', 'timestamp', 'price']).sort_values(['symbol', 'timestamp'])
deposit_plot = dfd[dfd.timestamp < '2021-05-23']

In [211]:
deposit_plot.head()

,symbol,timestamp,price,variable,value
0,ETH,2021-02-27 14:23:15+00:00,1479.770833,deposit,20.0
76,ETH,2021-02-27 14:23:15+00:00,1479.770833,debt,0.0
152,ETH,2021-02-27 14:23:15+00:00,1479.770833,debt_deposit,0.0
228,ETH,2021-02-27 14:23:15+00:00,1479.770833,non_debt_deposit,20.0
1,ETH,2021-02-27 22:11:42+00:00,1496.195833,deposit,24.0


In [207]:
base = alt.Chart(deposit_plot).encode(
    x=alt.X('timestamp', scale=alt.Scale(domain=['2021-02-27', '2021-05-23'])),
    y='value',
    color=alt.Color('variable', scale=scale),
).properties(
    width = 800,
    height=200
)

chart = alt.hconcat()
for symbol in ['ETH', 'WBTC']:
    chart &= base.transform_filter({'and': [alt.FieldEqualPredicate(field='symbol', equal=symbol),
                        alt.FieldOneOfPredicate(field='variable', oneOf=['debt_deposit', 'non_debt_deposit'])]}
                        ).mark_area() + base.transform_filter({'and': [alt.FieldEqualPredicate(field='symbol', equal=symbol),
                        alt.FieldOneOfPredicate(field='variable', oneOf=['deposit'])]}
                        ).mark_line().properties(
                            title=symbol
                        )
    


chart.properties(
    title=alt.Title('Total Cumulative Deposited Tokens', 
                    subtitle='Wallet for: 0xe47d5cc33517d5b8433ff249efe094b989989acc', 
                    fontSize=24, 
                    subtitleFontSize=16)
    ).configure_title(
        anchor='middle'
    )

alt.VConcatChart(...)

In [237]:
dfc = deposit_df.copy()
dfc['debt_deposit'] = dfc[['deposit', 'debt']].min(axis=1)
dfc['timestamp'] = pd.to_datetime(dfc.timestamp)
dfc['non_debt_deposit'] = dfc['deposit'] - dfc['debt_deposit']
dfc = dfc.melt(id_vars=['timestamp', 'price', 'symbol']).query("variable != 'debt'")
dfc['price_value'] = dfc['price']*dfc['value']
dfc['value'] = dfc.groupby('variable')['price_value'].cumsum()

In [241]:
base = alt.Chart(dfc[dfc.timestamp < '2021-05-23'].query("variable != 'deposit'")).encode(
    x=alt.X('timestamp', scale=alt.Scale(domain=['2021-02-27', '2021-05-23'])),
    y='value',
    color=alt.Color('variable'),
).properties(
    width = 800,
    height=200
).mark_area()

base

alt.Chart(...)

In [6]:
# Defining a swap
# Transfer in/out within same tx

# Check if 

def check_swap(wallet, tx_hash):
    '''
    Returns tx_hash if it is a swap
    '''
    
    transfers = df.query("tx_hash == @tx_hash").copy()
    
    if len(transfers) >= 2:

        if (account in transfers.sender.values) & (account in transfers.receiver.values) & ('deposit' not in transfers.tx_type.values):
            #print('Swap tx_hash:', tx_hash)
            transfers = df.query("tx_hash == @tx_hash").copy()
            block_time = transfers.tx_block_time.unique()[0]
            
            outgoing = transfers.query("sender == @account").drop_duplicates(subset=['symbol', 'amount'])[['symbol', 'amount']].iloc[0]
            incoming = transfers.query("receiver == @account").drop_duplicates(subset=['symbol', 'amount'])[['symbol', 'amount']].iloc[0]
            
            wallet.init_token(outgoing['symbol'])
            wallet.init_token(incoming['symbol'])
            
            # Fetch pre-updated balance and debt for each token
            out_token = wallet.get_token(outgoing['symbol'])
            in_token = wallet.get_token(incoming['symbol'])

            out_balance = out_token._get_previous_balance()
            out_debt = out_token._get_previous_debt()
            
            
            debt_ratio = min(round(out_debt/out_balance, 2), 1.0)
            in_token.update_debt(block_time, debt_ratio*incoming['amount'])
            out_token.update_debt(block_time, -debt_ratio*outgoing['amount'])
            #need tp reduce debt amounmt for out token
            return tx_hash
            
            

Balance at time $t$: WBTC = 10, USDC = 100, Debt at time $t$: WBTC = 10, USDC = 0

Swap 10 WBTC for 100 more USDC. Check debt ratio for outgoing token which is equal to 1. 
Update USDC debt based on total_incoming*debt_ratio

In [268]:
wallet.get_token('TUSD')

Token(token_symbol='TUSD', balance=OrderedDict([('2021-02-28 21:45:10.000 UTC', 689214.3017130159), ('2021-02-28 21:48:08.000 UTC', 0.0), ('2021-02-28 21:57:53.000 UTC', 689214.799312391), ('2021-02-28 22:02:59.000 UTC', 0.0), ('2021-02-28 22:10:16.000 UTC', 670413.0243259597), ('2021-02-28 22:12:27.000 UTC', 0.0), ('2021-02-28 22:24:44.000 UTC', 462810.5939110957), ('2021-02-28 22:27:04.000 UTC', 0.0)]), debt=OrderedDict([('2021-02-28 21:45:10.000 UTC', 689214.3017130159), ('2021-02-28 21:57:53.000 UTC', 689214.3017130159), ('2021-02-28 22:02:59.000 UTC', -0.49759937508497387), ('2021-02-28 22:10:16.000 UTC', 670412.5267265846), ('2021-02-28 22:12:27.000 UTC', -0.49759937508497387), ('2021-02-28 22:24:44.000 UTC', 462810.0963117206), ('2021-02-28 22:27:04.000 UTC', -0.49759937508497387)]), tx_history=[])

In [271]:
df.query("symbol == 'TUSD'")[['tx_block_time','sender', 'receiver', 'amount', 'tx_type']]

,tx_block_time,sender,receiver,amount,tx_type
33,2021-02-28 21:45:10.000 UTC,0x101cc05f4a51c0319f570d5e146a8c625198e636,0xe47d5cc33517d5b8433ff249efe094b989989acc,6.892143e+05,borrow
34,2021-02-28 21:48:08.000 UTC,0xe47d5cc33517d5b8433ff249efe094b989989acc,0x101cc05f4a51c0319f570d5e146a8c625198e636,6.892143e+05,deposit
36,2021-02-28 21:57:53.000 UTC,0x101cc05f4a51c0319f570d5e146a8c625198e636,0xe47d5cc33517d5b8433ff249efe094b989989acc,6.892148e+05,withdraw
38,2021-02-28 22:02:59.000 UTC,0xe47d5cc33517d5b8433ff249efe094b989989acc,0x74de5d4fcbf63e00296fd95d33236b9794016631,6.892148e+05,swap
42,2021-02-28 22:10:16.000 UTC,0x101cc05f4a51c0319f570d5e146a8c625198e636,0xe47d5cc33517d5b8433ff249efe094b989989acc,6.704130e+05,borrow
43,2021-02-28 22:12:27.000 UTC,0xe47d5cc33517d5b8433ff249efe094b989989acc,0x74de5d4fcbf63e00296fd95d33236b9794016631,6.704130e+05,swap
47,2021-02-28 22:24:44.000 UTC,0x101cc05f4a51c0319f570d5e146a8c625198e636,0xe47d5cc33517d5b8433ff249efe094b989989acc,4.628106e+05,borrow
48,2021-02-28 22:27:04.000 UTC,0xe47d5cc33517d5b8433ff249efe094b989989acc,0x74de5d4fcbf63e00296fd95d33236b9794016631,4.628106e+05,NaN
1282,2021-05-23 20:10:21.000 UTC,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xe47d5cc33517d5b8433ff249efe094b989989acc,1.806309e+06,NaN
1285,2021-05-23 20:13:05.000 UTC,0xe47d5cc33517d5b8433ff249efe094b989989acc,0x101cc05f4a51c0319f570d5e146a8c625198e636,1.806309e+06,repay


In [250]:
df['amount_usd'] = df.amount.multiply(df.usd_price)

In [274]:
1/1000000

1e-06

In [312]:
alt.Chart(df.query("tx_type == 'borrow'"), title='Total amount borrowed, by token').mark_bar().encode(
    y=alt.Y('symbol', title='Token').sort('-x'),
    x=alt.X('x:Q', title='$ USD (millions)').axis(format=".0f")
).transform_aggregate(
    total_usd = 'sum(amount_usd)',
    groupby=['symbol']
).transform_calculate(
    x='datum.total_usd/1000000'
).properties(
    width=400
)



alt.Chart(...)

In [388]:
df[df.symbol.str[0].str.lower() != 'a'].query("tx_type == 'deposit'")

alt.Chart(df[df.symbol.str[0].str.lower() != 'a'].query("tx_type == 'deposit'"), 
          title='Total amount deposited, by token').mark_bar().encode(
    y=alt.Y('symbol', title='Token').sort('-x'),
    x=alt.X('x:Q', title='$ USD (millions)').axis(format=".0f")
).transform_aggregate(
    total_usd = 'sum(amount_usd)',
    groupby=['symbol']
).transform_calculate(
    x='datum.total_usd/1000000'
).properties(
    width=400
)

alt.Chart(...)

In [356]:
import numpy as np

In [430]:
dfc= df.groupby(['tx_hash', 'sender', 'receiver', 'symbol', 'tx_type'])[['amount_usd', 'amount']].sum().reset_index().copy()

In [431]:
dfc.groupby('tx_hash').filter(lambda x: (len(x) > 2))

,tx_hash,sender,receiver,symbol,tx_type,amount_usd,amount
67,0x15f1c67b423db1b5a9b61890cb3551f305f8111328ec...,0x0000000000000000000000000000000000000000,0xe47d5cc33517d5b8433ff249efe094b989989acc,aWBTC,withdraw,0.000000e+00,2.476670e-13
68,0x15f1c67b423db1b5a9b61890cb3551f305f8111328ec...,0x3dfd23a6c5e8bbcfc9581d2e864a68feb6a076d3,0xe47d5cc33517d5b8433ff249efe094b989989acc,WBTC,withdraw,2.155326e+05,3.930103e+00
69,0x15f1c67b423db1b5a9b61890cb3551f305f8111328ec...,0xe47d5cc33517d5b8433ff249efe094b989989acc,0x0000000000000000000000000000000000000000,aWBTC,withdraw,0.000000e+00,3.930103e-10
259,0x59f9fe0c2c20b6771df5ed8e4ac7f6c0f674b2611189...,0x0000000000000000000000000000000000000000,0xe47d5cc33517d5b8433ff249efe094b989989acc,aETH,withdraw,0.000000e+00,4.512078e-02
260,0x59f9fe0c2c20b6771df5ed8e4ac7f6c0f674b2611189...,0x3dfd23a6c5e8bbcfc9581d2e864a68feb6a076d3,0xe47d5cc33517d5b8433ff249efe094b989989acc,ETH,withdraw,1.025980e+06,4.000000e+02
261,0x59f9fe0c2c20b6771df5ed8e4ac7f6c0f674b2611189...,0xe47d5cc33517d5b8433ff249efe094b989989acc,0x0000000000000000000000000000000000000000,aETH,withdraw,0.000000e+00,4.000000e+02
383,0x7c1821650ca5b8ece2c8a5f42c7af5411c55dc3dae85...,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xe47d5cc33517d5b8433ff249efe094b989989acc,USDC,swap,1.135151e-03,1.133000e-03
384,0x7c1821650ca5b8ece2c8a5f42c7af5411c55dc3dae85...,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xe47d5cc33517d5b8433ff249efe094b989989acc,WBTC,swap,2.477536e+05,4.411649e+00
385,0x7c1821650ca5b8ece2c8a5f42c7af5411c55dc3dae85...,0xe47d5cc33517d5b8433ff249efe094b989989acc,0x74de5d4fcbf63e00296fd95d33236b9794016631,USDC,swap,2.504746e+05,2.500000e+05
429,0x8d5cce6fecde55f553aa6000430431fe6dcb4719d4b6...,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xe47d5cc33517d5b8433ff249efe094b989989acc,USDC,swap,6.864909e-04,6.860000e-04


In [435]:
swap_df = dfc.query("amount_usd > 1").groupby('tx_hash').filter(lambda x: (len(x) >= 2) & (account in x['sender'].values) & (account in x['receiver'].values) & (('swap' in x['tx_type'].values) | (np.nan in x['tx_type'].values)))

swap_df['direction'] = np.where(swap_df.sender == account, 'sell', 'buy')
swap_df = swap_df.pivot(index=['tx_hash'], columns=['direction'], values=['amount_usd', 'amount', 'symbol']).reset_index()
swap_df.columns = ['tx_hash', 'buy_usd', 'sell_usd', 'buy_amount', 'sell_amount', 'buy_symbol', 'sell_symbol']
swap_df.head()

,tx_hash,buy_usd,sell_usd,buy_amount,sell_amount,buy_symbol,sell_symbol
0,0x00c5eed483dc4629107502a047ed6cf1809365e4cff6...,197605.100967,200165.316667,3.59363,200000.0,WBTC,DAI
1,0x01b7427e073d089cd58cbd6bd9f549c5711801ebddcb...,492745.303227,500728.791667,8.985208,500000.0,WBTC,USDC
2,0x0202ca13f90c91e5f99e77c1deff4a983ebee12cec0c...,99445.41468,100192.441667,1.713664,100000.0,WBTC,USDT
3,0x02438ed8b6e89664e35ac0291db8f09081ef82c397ac...,150833.279776,150126.8875,58.1822,150000.0,ETH,USDC
4,0x0515e28b9888095a7781c69056c4c2c8fb6503803a07...,198104.602565,201005.016667,3.631814,200000.0,WBTC,USDT


In [436]:
swap_pairs = swap_df.groupby(['buy_symbol', 'sell_symbol'], as_index=False)[['buy_usd']].sum()
swap_pairs['pair'] = swap_pairs['buy_symbol'] + '-' + swap_pairs['sell_symbol']

In [437]:
alt.Chart(swap_pairs, 
          title='Total amount swapped, by token pairing').mark_bar().encode(
    y=alt.Y('pair', title='Token Pair (buy - sell)').sort('-x'),
    x=alt.X('x:Q', title='$ USD (millions)').axis(format=".1f")
).transform_calculate(
    x='datum.buy_usd/1000000'
).properties(
    width=400
)

alt.Chart(...)

In [408]:
df.query("symbol == 'WBTC'").groupby('tx_type')['amount_usd'].sum()

tx_type
deposit     7.138681e+07
swap        1.179077e+08
withdraw    1.274539e+07
Name: amount_usd, dtype: float64